In [ ]:
# Copyright 2022 Sony Semiconductor Solutions Corp. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Imports

In [ ]:
import errno
import json
import jsonschema
import os
import shutil
import time
from pathlib import Path
from cvat_sdk.api_client import ApiClient, Configuration

## Load Configurations

Load the configuration file and set the variables.

In [ ]:
def validate_symlink(path: Path):
    if path.is_symlink():
        msg = "Symbolic link is not supported. Please use real folder or file"
        raise OSError(errno.ELOOP, f"{msg}", f"{path}")


configuration_path = Path("./configuration.json")
validate_symlink(configuration_path)

with open(configuration_path, "r") as f:
    app_configuration = json.load(f)

configuration_schema_path = Path("./configuration_schema_export.json")
validate_symlink(configuration_schema_path)

with open(configuration_schema_path, "r") as f:
    json_schema = json.load(f)

# Validate configuration.
jsonschema.validate(app_configuration, json_schema)

cvat_username = app_configuration["cvat_username"]
cvat_password = app_configuration.get("cvat_password", "")
cvat_project_id = app_configuration.get("cvat_project_id", "")

export_format = app_configuration["export_format"]
export_dir = app_configuration["export_dir"].replace(os.path.sep, "/")
validate_symlink(Path(export_dir))

## Set Authentication

In [ ]:
# Set up an API client
configuration = Configuration(
    host="http://localhost:8080/",
    username=cvat_username,
    password=cvat_password,
)

## Download Dataset
Export dataset from the CVAT project.

In [ ]:
with ApiClient(configuration) as api_client:
    # str | Used to start downloading process after annotation file had been created (optional)
    action = "download"
    format = export_format

    status = 0
    temp_filepath = ""
    (projectslist, response) = api_client.projects_api.list()
    id = cvat_project_id
    filename = "project_id_" + str(cvat_project_id) + ".zip"
    wt_list = [".", "..", "...", "...."]
    wt_len = len(wt_list)
    wt_count = 0

    while status != 200 and status != 201:
        (data, response) = api_client.projects_api.retrieve_dataset(
            id,
            action=action,
            filename=filename,
            format=format,
        )
        status = response.status
        if status != 200 and status != 201:
            print("\rDataset is downloading now." + wt_list[wt_count], end="")
        else:
            print("\nDataset is downloaded.")
        time.sleep(1)
        temp_filepath = data.name
        wt_count += 1
        if wt_count == wt_len:
            wt_count = 0

    if not os.path.exists(export_dir):
        os.makedirs(export_dir)
    if not os.path.exists(export_dir + filename):
        shutil.move(temp_filepath, export_dir)
        print("Dataset moved. Download process is completed.")
    else:
        raise FileExistsError(
            f"Failed to move dataset. A dataset file with the same name exists in the export_dir: {export_dir}"
        )